In [8]:
include("../ToolBox/ToolBox.jl")
include("../path.jl")
using .ToolBox
using .Geometry
using .Load
using .SlcUtil
import .Misc
using DelimitedFiles
using Statistics

In [ ]:
# for saving csv's
function save_mean_data(data, name)
    row_means = mean(data, dims=2);
    writedlm(name*"_real.csv", real.(row_means));
    writedlm(name*"_imag.csv", imag.(row_means));
end

### Data paths

In [5]:
# Approximate view for 2 burst in master
master_view = [1551:3019, 946:24513]
meta = (Load.slc_meta(master_meta_path), Load.slc_meta(slave_meta_path))
precise_orbit = (Load.precise_orbit(master_pod_path, meta[1]["t_0"]), 
                 Load.precise_orbit(slave_pod_path, meta[2]["t_0"]))

# load DEM
footprint = SlcUtil.footprint(meta[1], master_view)
latlon_window = ((minimum(footprint[1]),maximum(footprint[1])),(minimum(footprint[2]),maximum(footprint[2])))
dem = Load.dem(dem_path, latlon_window; nan_fill= 0, padding=[90,90]);

proj_create: init=epsg:/init=IGNF: syntax not supported in non-PROJ4 emulation mode


In [9]:
stride=(2,8)

# get some coficents
c = 299792458
range_pixel_spacing =  c/(2*meta[1]["range_sampling_rate"])
lambda =  c/meta[1]["radar_frequency"]

# Get the midburst speed
v_mid = mid_burst_speed(precise_orbit[2], meta[2]);

# look_up_table
mosaic_view = SlcUtil.mosaic_view(meta[1],master_view)
lut = look_up_table(mosaic_view,meta,precise_orbit,dem,stride=(2,8))

# get master line and sample
master_line, master_sample = Misc.flatten(mosaic_view...)

# interpolate slave line and sample
slave_line = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
reshape(lut["slave_line"],(length(lut["master_line"]),length(lut["master_sample"])))
,mosaic_view[1], mosaic_view[2])
slave_sample = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
    reshape(lut["slave_sample"],(length(lut["master_line"]),length(lut["master_sample"])))
    ,mosaic_view[1], mosaic_view[2]);

slave_line = reshape(slave_line,:)
slave_sample= reshape(slave_sample,:);

# master bursts
master_start_burst = ceil(Int,(master_view[1].start)/meta[1]["lines_per_burst"])
master_end_burst = ceil(Int,(master_view[1].stop-1)/meta[1]["lines_per_burst"])

# find the slave bursts
slave_start_line = minimum(slave_line)
slave_start_burst = sum([slave_start_line > elem ? 1 : 0 for elem in meta[2]["burst_meta"]["first_line_mosaic"]])

## Get the number of lines in the first selected burst
lines_in_first_burst_master = master_start_burst*meta[1]["lines_per_burst"] - master_view[1].start
lines_in_first_burst_slave = meta[2]["burst_meta"]["first_line_mosaic"][slave_start_burst]+meta[2]["lines_per_burst"] - slave_start_line

## Check if the number of lines are approximately the same
if lines_in_first_burst_slave > lines_in_first_burst_master + 100
    # the number of lines are not the same. The slave burst before should be included
    slave_start_burst = slave_start_burst -1
    lines_in_first_burst_slave = slave_start_burst*meta[2]["lines_per_burst"] - slave_start_line
end

# compute the offset between the burst number in slave and master
delta_burst = slave_start_burst - master_start_burst

## check if it is in slave image
slave_end_burst = master_end_burst + delta_burst

if slave_start_burst == 0
    println("Warning: start line not in slave image")
end

if slave_end_burst > meta[2]["burst_count"]
    println("Warning: end line not in slave image")
end

if minimum(slave_sample) < 1
    println("Warning: start sample not in slave image")
end

if maximum(slave_sample) > meta[2]["samples_per_burst"]
    println("Warning: end sample not in slave image")
end

# Initialize arrays for the results
coreg_slave = Array{Complex{Float64}}(undef,length.(master_view)...)
flat_inferogram = Array{Complex{Float64}}(undef,length.(master_view)...)

# load and resample one burst at a time
for n_master in master_start_burst:master_end_burst

    n_slave =  n_master + delta_burst

    start_line_master = meta[1]["lines_per_burst"] * (n_master-1)+1
    end_line_master = meta[1]["lines_per_burst"] * n_master

    ## offset cused by overlap between burts
    over_lap_master = meta[1]["lines_per_burst"]*(n_master-1)+1 - meta[1]["burst_meta"]["first_line_mosaic"][n_master]
    over_lap_slave = meta[2]["lines_per_burst"]*(n_slave-1)+1 - meta[2]["burst_meta"]["first_line_mosaic"][n_slave]

    if n_master == master_start_burst
        start_line_master = master_view[1].start
    end

    if n_master == master_end_burst
        end_line_master = master_view[1].stop
    end

    # finds the lines in the burst
    in_burst = (start_line_master-over_lap_master) .<= master_line .<= (end_line_master-over_lap_master)


    # get lines
    master_line_n = master_line[in_burst] .+ over_lap_master
    slave_line_n = slave_line[in_burst] .+ over_lap_slave

    # get samples
    master_sample_n = master_sample[in_burst]
    slave_sample_n = slave_sample[in_burst]

    # load data
    slave_view_n = round(Int,minimum(slave_line_n)): round(Int,maximum(slave_line_n)),
                round(Int,minimum(slave_sample_n)): round(Int,maximum(slave_sample_n))
    slave_data = Load.slc_data(slave_data_path, slave_view_n);
    save_mean_data(slave_data, "original")
    
    # deramp
    phi = SlcUtil.phase_ramp(Misc.flatten(slave_view_n...)..., n_slave, meta[2], v_mid[n_slave]);
    slave_data = slave_data .* reshape(exp.(-phi .* im), size(slave_data));
    save_mean_data(slave_data, "deramped")
    
    # dimension of resample
    dim = (convert(Int,length(master_line_n)/length(master_view[2])),length(master_view[2]))

    #resample
    slave_data = Misc.resample(slave_view_n,slave_data,slave_line_n,slave_sample_n)
    slave_data = reshape(slave_data, dim);

    # reramp
    phi = SlcUtil.phase_ramp(slave_line_n, slave_sample_n, n_slave, meta[2], v_mid[n_slave])
    slave_data = slave_data .* reshape(exp.(phi .* im), dim);
    save_mean_data(slave_data, "reramped")
    save_mean_data(reshape(exp.(phi .* im), dim), "chirp")

    # flat_inferogram
    flat = exp.(4*pi.*(master_sample_n.-slave_sample_n).*range_pixel_spacing./lambda.*im)
    flat = reshape(flat,dim);

    # store results
    coreg_slave[(start_line_master:end_line_master) .- (master_view[1].start-1),:] .= slave_data
    flat_inferogram[(start_line_master:end_line_master) .- (master_view[1].start-1),:] .= flat
end

# remove the slave goemtry and return the dict as geo_ref_table
delete!(lut, "slave_sample")
delete!(lut, "slave_line")
geo_ref_table = lut;

(1469, 23567)

proj_create: init=epsg:/init=IGNF: syntax not supported in non-PROJ4 emulation mode


Dict{String,Any} with 5 entries:
  "master_sample" => [946, 954, 962, 970, 978, 986, 994, 1002, 1010, 1018  …  2…
  "latitude"      => [56.5054, 56.5052, 56.5049, 56.5047, 56.5044, 56.5042, 56.…
  "heights"       => [49.2302, 48.7196, 47.9499, 46.5236, 45.2859, 44.0984, 30.…
  "master_line"   => [1369, 1371, 1373, 1375, 1377, 1379, 1381, 1383, 1385, 138…
  "longitude"     => [8.58565, 8.58559, 8.58552, 8.58547, 8.58542, 8.58537, 8.5…

In [14]:
# If you want to see the data
# master_data = Load.slc_data(master_data_path,master_view);
# SlcUtil.plot_phase(master_data.* conj.(coreg_slave) .* flat_inferogram)

proj_create: init=epsg:/init=IGNF: syntax not supported in non-PROJ4 emulation mode
